# **LinkedIn Job Scraper for Panama City**

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import re

# Function to generate LinkedIn job search URL with pagination
def obtener_url(position, location, start=0):
    return f"https://www.linkedin.com/jobs/search/?keywords={position}&location={location}&start={start}"

# Function to clean extracted text
def limpiar(texto):
    return re.sub(r'\s+', ' ', texto).strip() if texto else 'Not available'

# Function to extract job details safely
def obtener_data(job):
    try:
        location = limpiar(job.find('span', class_='job-search-card__location').get_text())
        title = limpiar(job.find('h3', class_='base-search-card__title').get_text())
        company = limpiar(job.find('h4', class_='base-search-card__subtitle').get_text())

        # Extract job URL
        job_link = job.find('a', class_='base-card__full-link')
        job_url = limpiar(job_link.get('href')) if job_link else 'Not available'

        return {
            "Location": location,
            "Job Title": title,
            "Company": company,
            "URL": job_url
        }
    except Exception as e:
        print(f"Error extracting job details: {e}")
        return None

# Function to scrape LinkedIn jobs for a given city
def codigo_principal(position, location):
    lista_trabajos = []
    start = 0

    while True:
        url = obtener_url(position, location, start)

        # Headers to avoid blocking
        headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 "
                          "(KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36"
        }

        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            break  # Stop if request fails

        soup = BeautifulSoup(response.text, 'html.parser')

        # Find job list
        joblist = soup.find('ul', class_='jobs-search__results-list')
        if not joblist:
            print(f"No more jobs found. Stopping pagination.")
            break

        all_jobs = joblist.find_all('li')

        # If no jobs are found on this page, stop
        if len(all_jobs) == 0:
            break

        print(f"Found {len(all_jobs)} jobs on page {start // 25 + 1}")

        for job in all_jobs:
            job_data = obtener_data(job)
            if job_data:
                lista_trabajos.append(job_data)

        # Stop if the number of jobs is less than 25 (indicating last page)
        if len(all_jobs) < 25:
            break

        start += 25  # Move to next page
        time.sleep(3)  # Prevent getting blocked

    return lista_trabajos

# Run the scraper for a specific city
position = "data scientist"
location = "Panama City, Panamá, Panama"
trabajos = codigo_principal(position, location)
